<a href="https://colab.research.google.com/github/aissahm/Unicamp-MO431/blob/master/4o_Exerc%C3%ADcio_de_casa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **4o Exercício de casa**

**Objective:**

Seearch for the best combinations of hyperparameters for the SVM for regression in a particular database


In [0]:
# Imports
import numpy as np

X_train = np.load("/content/Xtreino5.npy")
Y_train = np.load("/content/ytreino5.npy")

X_test = np.load("/content/Xteste5.npy")
Y_test = np.load("/content/yteste5.npy")

In [4]:
print(X_train.shape)
print(Y_train.shape)

(404, 13)
(404,)


In [22]:
from sklearn.svm import SVR
import numpy as np

svr = SVR(kernel='rbf', C=10.0, epsilon=3.5, gamma=1.)
svr.fit(X_train, Y_train)

SVR(C=10.0, cache_size=200, coef0=0.0, degree=3, epsilon=3.5, gamma=1.0,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [0]:
Y_pred = svr.predict(X_test)

In [23]:
print(svr.score(X_test, Y_test))

-0.0008088086106268211


# **SVM Regressor**

We will use the SVM from scikit-learn : https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html

We will use the "rbf" kernel with three parameters: C, gamma, and epsilon.
 
With:

1. C entre 2^{-5} e 2^{15} (uniforme nos expoentes)
2. gamma entre 2^{-15} e 2^{3} (uniforme nos expoentes)
3. epsilon entre 0.05 a 1.0 (uniforme neste intervalo)

**Medida de erro**

We will train the SVM on training data, with the combination of the parameters, and then measure the medida de erro with the test data.

# **Random search**

In [31]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

from sklearn.svm import SVR
import numpy as np

C_lowerbound = 2**-5
C_upperbound = 2**15

gamma_lowerbound = 2**-15
gamma_upperbound = 2**3

svr = SVR(kernel='rbf')
distributions = dict(C= uniform(loc=C_lowerbound, scale=C_upperbound), gamma= uniform(loc=gamma_lowerbound, scale=gamma_upperbound), epsilon=uniform())
svr_search = RandomizedSearchCV(svr, distributions, random_state=0)
search = svr_search.fit(X_train, Y_train)
search.best_params_

{'C': 18613.715427925974,
 'epsilon': 0.925596638292661,
 'gamma': 0.5683189831612205}

In [33]:
svr = SVR(kernel='rbf', C=search.best_params_["C"], epsilon=search.best_params_["epsilon"], gamma=search.best_params_["gamma"])
svr.fit(X_train, Y_train)

SVR(C=18613.715427925974, cache_size=200, coef0=0.0, degree=3,
    epsilon=0.925596638292661, gamma=0.5683189831612205, kernel='rbf',
    max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [35]:
print("Error: ", svr.score(X_test, Y_test))

Error:  0.0006079865289774222


# **Grid Search**

In [0]:
from sklearn.model_selection import GridSearchCV
svr = GridSearchCV(SVR(kernel='rbf'), param_grid={"C": np.linspace(C_lowerbound, C_upperbound, num=5), "gamma": np.linspace(gamma_lowerbound, gamma_upperbound, num=5), "epsilon": np.linspace(0.0, 1.0, num=5)})

In [0]:
svr.fit(X_train, Y_train)
sv_ind = svr.best_estimator_.support_

In [0]:
best = svr.best_estimator_

In [48]:
print(best)

SVR(C=8192.0234375, cache_size=200, coef0=0.0, degree=3, epsilon=1.0,
    gamma=3.0517578125e-05, kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)


In [50]:
svr = SVR(kernel='rbf', C=best.C, epsilon=best.epsilon, gamma=best.gamma)
svr.fit(X_train, Y_train)
print("Error: ", svr.score(X_test, Y_test))

Error:  0.7948348497877594


# **Bayesian Optimization**

We use the library from: https://github.com/hyperopt/hyperopt

In [0]:
!pip install hyperopt

In [97]:
# define an objective function
def objective(params):
    C_value = params["C"] 
    gamma_value= params["gamma"] 
    epsilon_value  = params["epsilon"]
    C_value = 2**C_value
    gamma_value = 2**gamma_value
    svr = SVR(kernel='rbf', C=C_value, epsilon = epsilon_value, gamma= gamma_value)
    svr.fit(X_train, Y_train)
    return svr.score(X_test, Y_test)

C_lowerbound = 2**-5
C_upperbound = 2**15

gamma_lowerbound = 2**-15
gamma_upperbound = 2**3

# define a search space
from hyperopt import hp
space = {
    'C': hp.uniform('C', -5, 15),
    'gamma': hp.uniform('gamma', -15, 3),
    'epsilon': hp.uniform('epsilon', 0.0, 1)
}

# minimize the objective over the space
from hyperopt import fmin, tpe, space_eval
best = fmin(objective, space, algo=tpe.suggest, max_evals=100)

100%|██████████| 100/100 [00:04<00:00, 21.18it/s, best loss: -0.035097100915801205]


In [98]:
print(best)

{'C': -3.87990316232706, 'epsilon': 0.388098152552051, 'gamma': 0.34845060101124603}


In [100]:
svr = SVR(kernel='rbf', C=2**best["C"], epsilon = best["epsilon"], gamma=2** best["gamma"])
svr.fit(X_train, Y_train)
print(svr.score(X_test, Y_test))

-0.035097100915801205


# **PSO**

In [57]:
!pip install  pyswarm

  Created wheel for pyswarm: filename=pyswarm-0.6-cp36-none-any.whl size=4481 sha256=2b0900b49f88f03094780b30eb42c579fcf27523184074757d707313b8cb455b
  Stored in directory: /root/.cache/pip/wheels/37/c5/f6/b33b9ac00040cb95c1f00af982a4197334a672d6de43f4699f
Successfully built pyswarm


In [60]:
from pyswarm import pso

def objective(x):
    C_value, gamma_value, epsilon_value  = x
    C_value = 2**C_value
    gamma_value = 2**gamma_value
    svr = SVR(kernel='rbf', C= C_value, epsilon= epsilon_value, gamma= gamma_value)
    svr.fit(X_train, Y_train)
    return svr.score(X_test, Y_test)

C_lowerbound = -5
C_upperbound = 15

gamma_lowerbound = -15
gamma_upperbound = 3

epsilon_lowerbound = 0.0
epsilon_upperbound = 1.0

lb = [C_lowerbound, gamma_lowerbound, epsilon_lowerbound]
ub = [C_upperbound, gamma_upperbound, epsilon_upperbound]

xopt, fopt = pso(objective, lb, ub, swarmsize = 11, maxiter= 11)

Stopping search: maximum iterations reached --> 11


In [61]:
print( xopt, fopt )

[-4.98293722  0.34178512  0.38238927] -0.03578826966911808


In [59]:
print( xopt, fopt )

[0.03125    6.11680812 0.63745124] -0.033306864254991186


# **Simulated annealing**

In [62]:
!pip install simanneal

# **CMA**

In [0]:
pip install cma

In [116]:
import cma

def objective(x):
    #print("x", x)
    C_value = 2**( -5 + x[0]*20 )
    gamma_value = 2**( -15 + x[1]*18 )
    svr = SVR(kernel='rbf', C= C_value, epsilon=abs( x[2] ), gamma= gamma_value)
    svr.fit(X_train, Y_train)
    return svr.score(X_test, Y_test)

C_lowerbound = -5
C_upperbound = 15

gamma_lowerbound = -15
gamma_upperbound = 3

epsilon_lowerbound = 0.1
epsilon_upperbound = 1.0

es = cma.CMAEvolutionStrategy(3 * [0.1], 0.1, {'seed':125})
es.optimize(objective)

(3_w,7)-aCMA-ES (mu_w=2.3,w_1=58%) in dimension 3 (seed=125, Wed May  6 03:53:54 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      7 -2.731121310024110e-02 1.0e+00 1.23e-01  1e-01  2e-01 0:00.1
    2     14 -1.844965362025763e-02 1.5e+00 1.29e-01  1e-01  2e-01 0:00.3
    3     21 -3.137542277087824e-02 1.4e+00 2.02e-01  2e-01  2e-01 0:00.4
   30    210 -3.554120449827858e-02 2.2e+00 1.01e+00  6e-01  1e+00 0:03.4
   66    462 -3.611339830242355e-02 1.3e+02 1.28e-01  2e-03  2e-01 0:07.4
   85    595 -3.611339830242355e-02 9.0e+02 5.90e-02  1e-04  9e-02 0:09.7


In [117]:
es.result_pretty()

termination on tolflatfitness=1
final/bestever f-value = -3.611340e-02 -3.611340e-02
incumbent solution: [-2.9365390382292507, -0.5215112655250695, -0.3997571846835017]
std deviation: [0.030118214124302517, 0.08617297322283334, 0.00010328417095892205]


CMAEvolutionStrategyResult(xbest=array([-2.91915399, -0.86217398, -0.39980981]), fbest=-0.03611339830242355, evals_best=405, evaluations=595, iterations=85, xfavorite=array([-2.93653904, -0.52151127, -0.39975718]), stds=array([0.03011821, 0.08617297, 0.00010328]), stop={'tolflatfitness': 1})

In [118]:
C_value = 2**( -5 + -2.93*20 )
gamma_value = 2**( -15 +  -0.52*18 )

print(C_value, gamma_value)

7.153066717358898e-20 4.644182799223066e-08


In [119]:
svr = SVR(kernel='rbf', C= C_value, epsilon=abs( -0.399 ), gamma= gamma_value)
svr.fit(X_train, Y_train)
print( svr.score(X_test, Y_test))

-0.03606702067500023
